In [5]:
import pandas as pd

sample = "D:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\data\processed\Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet"
df = pd.read_parquet(sample, engine="pyarrow")
print(f"Loaded {len(df):,} rows × {df.shape[1]} columns for {sample}")


Loaded 1,048,575 rows × 80 columns for D:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\data\processed\Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet


Step 1: Exploratory Data Analysis (EDA)
Goals:

Understand how many classes are present in each file.

Identify which files contain both Benign and attack labels.

See if there’s class imbalance.



In [6]:
import pandas as pd
import os

data_dir = "D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/processed"
parquet_files = [f for f in os.listdir(data_dir) if f.endswith(".parquet")]

for pf in parquet_files:
    df = pd.read_parquet(os.path.join(data_dir, pf))
    print(f"File: {pf}")
    print(df['Label'].value_counts())
    print("-" * 40)


File: Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet
Label
Benign    762384
Attack    286191
Name: count, dtype: int64
----------------------------------------
File: Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet
Label
Attack    601803
Benign    446772
Name: count, dtype: int64
----------------------------------------
File: Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet
Label
Benign    1048009
Attack        566
Name: count, dtype: int64
----------------------------------------
File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet
Label
Benign    7372557
Attack     576191
Name: count, dtype: int64
----------------------------------------
File: Thursday-01-03-2018_TrafficForML_CICFlowMeter.parquet
Label
Benign    238037
Attack     93088
Name: count, dtype: int64
----------------------------------------
File: Thursday-15-02-2018_TrafficForML_CICFlowMeter.parquet
Label
Benign    996077
Attack     52498
Name: count, dtype: int64
----------------------------------------


In [ ]:
import os
import pandas as pd

# Path to Parquet files
parquet_dir = r"D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/processed"
parquet_files = [f for f in os.listdir(parquet_dir) if f.endswith(".parquet")]

print(f"Found {len(parquet_files)} Parquet files.\n")

for pf in sorted(parquet_files):
    fpath = os.path.join(parquet_dir, pf)
    df = pd.read_parquet(fpath)
    print(f"File: {pf}")
    print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"  Label counts:\n{df['Label'].value_counts(dropna=False)}")
    missing = df.isnull().sum()
    n_missing = missing[missing > 0]
    if not n_missing.empty:
        print(f"  Columns with missing values:\n{n_missing}")
    else:
        print("  No missing values found.")
    print("-" * 60)


Found 10 Parquet files.

File: Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet
  Shape: 1,048,575 rows × 80 columns
  Label counts:
Label
Benign    762384
Bot       286191
Name: count, dtype: int64
  Columns with missing values:
Flow Byts/s    2558
dtype: int64
------------------------------------------------------------
File: Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet
  Shape: 1,048,575 rows × 80 columns
  Label counts:
Label
DoS attacks-Hulk            461912
Benign                      446772
DoS attacks-SlowHTTPTest    139890
Label                            1
Name: count, dtype: int64
  Columns with missing values:
Dst Port         1
Protocol         1
Flow Duration    1
Tot Fwd Pkts     1
Tot Bwd Pkts     1
                ..
Active Min       1
Idle Mean        1
Idle Std         1
Idle Max         1
Idle Min         1
Length: 78, dtype: int64
------------------------------------------------------------
File: Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet
  Shap

In [ ]:
import os
import pandas as pd
import numpy as np

# Paths
PARQUET_DIR = 'D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/processed'
CLEANED_DIR = 'D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned'
os.makedirs(CLEANED_DIR, exist_ok=True)

# Columns to always drop if present
drop_cols = ['Timestamp', 'Flow ID', 'Src IP', 'Dst IP', 'Src Port', 'Label']  # 'Label' dropped after relabeling

# Harmonized set of feature columns 
feature_cols = None

for fname in sorted(os.listdir(PARQUET_DIR)):
    if not fname.endswith('.parquet'):
        continue
    fpath = os.path.join(PARQUET_DIR, fname)
    print(f"Processing {fname}...")
    df = pd.read_parquet(fpath)
    
    # 1. Remove rows 
    df = df[df['Label'].notna()]
    df = df[df['Label'] != 'Label']
    
    # 2. Standardize labels: Benign=0, attack=1
    df['Label'] = df['Label'].apply(lambda x: 0 if str(x).strip().lower() == 'benign' else 1)
    
    # 3. Drop unneeded columns (keep 'Label')
    cols_to_drop = [col for col in drop_cols if col != 'Label' and col in df.columns]
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop, errors='ignore')
    
    # 4. Set/verify harmonized columns
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col != 'Label']
    else:
        missing = [c for c in feature_cols if c not in df.columns]
        extra = [c for c in df.columns if c not in feature_cols and c != 'Label']
        # Add missing with NaN, drop extras
        if missing:
            for m in missing:
                df[m] = np.nan
        if extra:
            df = df.drop(columns=extra, errors='ignore')
        df = df[feature_cols + ['Label']]
    
    # 5. Convert all feature columns to float
    for col in feature_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # 6. Fill missing values
    df[feature_cols] = df[feature_cols].fillna(df[feature_cols].median())
    
    # 7. Save cleaned parquet
    out_path = os.path.join(CLEANED_DIR, fname)
    df.to_parquet(out_path, index=False)
    print(f"Saved cleaned file: {out_path} (Rows: {len(df)})")


print("All files cleaned and harmonized for binary classification!")


Processing Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet...
Saved cleaned file: D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned\Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet (Rows: 1048575)
Processing Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet...
Saved cleaned file: D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned\Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet (Rows: 1048574)
Processing Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet...
Saved cleaned file: D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned\Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet (Rows: 1048575)
Processing Thuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet...
Saved cleaned file: D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned\Thuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet (Rows: 7948748)
Processing Thursday-01-03-2018_T

In [24]:
import pandas as pd
import glob

cleaned_dir = "D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned"
file_list = sorted(glob.glob(f"{cleaned_dir}/*.parquet"))

# Aggregate for overall class balance and missingness
total_counts = pd.Series(dtype=int)
total_missing = pd.Series(dtype=int)

for path in file_list:
    df = pd.read_parquet(path)
    print(f"{path.split('/')[-1]} - Label counts:\n{df['Label'].value_counts()}\n")
    total_counts = total_counts.add(df['Label'].value_counts(), fill_value=0)
    missing = df.isnull().sum()
    print(f"Missing values:\n{missing[missing>0]}\n")
    total_missing = total_missing.add(missing, fill_value=0)

print("=== OVERALL LABEL COUNTS ===")
print(total_counts)
print("\n=== OVERALL MISSING VALUES (any feature) ===")
print(total_missing[total_missing > 0])


cleaned\Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet - Label counts:
Label
0    762384
1    286191
Name: count, dtype: int64

Missing values:
Series([], dtype: int64)

cleaned\Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet - Label counts:
Label
1    601802
0    446772
Name: count, dtype: int64

Missing values:
Series([], dtype: int64)

cleaned\Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet - Label counts:
Label
0    1048009
1        566
Name: count, dtype: int64

Missing values:
Series([], dtype: int64)

cleaned\Thuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet - Label counts:
Label
0    7372557
1     576191
Name: count, dtype: int64

Missing values:
Series([], dtype: int64)

cleaned\Thursday-01-03-2018_TrafficForML_CICFlowMeter.parquet - Label counts:
Label
0    238037
1     93063
Name: count, dtype: int64

Missing values:
Series([], dtype: int64)

cleaned\Thursday-15-02-2018_TrafficForML_CICFlowMeter.parquet - Label counts:
Label
0    996077
1     52498
Name: cou

In [ ]:
import pyarrow.parquet as pq
import lightgbm as lgb
import numpy as np
import os
from glob import glob
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# === CONFIGURATION ===
CLEANED_DIR = r"D:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\data\cleaned"
MODEL_PATH = "final_lightgbm.txt"
BATCH_SIZE = 100_000

# 1. Auto-detect all cleaned parquet files, split train/test (last one as test for demo)
file_list = sorted(glob(os.path.join(CLEANED_DIR, "*.parquet")))
train_files = file_list[:-1]
test_file = file_list[-1]

print("Train files:", train_files)
print("Test file:", test_file)

# === CALCULATE scale_pos_weight ===
print("\nCalculating scale_pos_weight...")
total_0, total_1 = 0, 0
for f in train_files:
    parquet = pq.ParquetFile(f)
    for batch in parquet.iter_batches(batch_size=BATCH_SIZE, columns=['Label']):
        arr = batch.to_pandas()['Label'].value_counts()
        total_0 += arr.get(0, 0)
        total_1 += arr.get(1, 0)

scale_pos_weight = total_0 / total_1
print(f"scale_pos_weight: {scale_pos_weight:.3f}")

# 2. Pick feature columns (from any file)
sample = pq.read_table(train_files[0], columns=None).to_pandas().iloc[:1]
feature_cols = [col for col in sample.columns if col != 'Label']

# 3. LightGBM parameters
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'verbose': -1,
    'scale_pos_weight': scale_pos_weight
}

# === TRAINING ===
booster = None
for file_path in train_files:
    print(f"\nProcessing: {file_path}")
    parquet_file = pq.ParquetFile(file_path)
    batch_idx = 1
    for batch in parquet_file.iter_batches(batch_size=BATCH_SIZE):
        df = batch.to_pandas()
        X = df[feature_cols].astype(np.float32)
        y = df['Label'].astype(np.int32)
        lgb_train = lgb.Dataset(X, y, free_raw_data=True)
        if booster is None:
            print("  Training new model …")
            booster = lgb.train(params, lgb_train, num_boost_round=100)
        else:
            print(f"  Continuing training … (batch {batch_idx})")
            booster = lgb.train(params, lgb_train, num_boost_round=100, init_model=booster)
        del df, X, y, lgb_train  # Free memory
        batch_idx += 1

booster.save_model(MODEL_PATH)
print("\nModel saved as:", MODEL_PATH)

# === EVALUATION ===
print(f"\nEvaluating on: {test_file}")
test_parquet = pq.ParquetFile(test_file)
y_true_all, y_pred_prob = [], []
for batch in test_parquet.iter_batches(batch_size=BATCH_SIZE):
    df = batch.to_pandas()
    X = df[feature_cols].astype(np.float32)
    y = df['Label'].astype(np.int32)
    y_prob = booster.predict(X)
    y_true_all.extend(y.tolist())
    y_pred_prob.extend(y_prob.tolist())
    del df, X, y, y_prob

# --- Threshold sweep
thresholds = [0.5, 0.4, 0.3, 0.2]
for threshold in thresholds:
    y_pred = (np.array(y_pred_prob) > threshold).astype(int)
    print(f"\n=== Results for threshold={threshold} ===")
    print("Accuracy:", accuracy_score(y_true_all, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true_all, y_pred))
    print("Classification Report:\n", classification_report(y_true_all, y_pred, digits=4))

print("\nDone! The model and thresholded evaluation results are printed.")


Train files: ['D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thursday-01-03-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thursday-15-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\

### Updated Lightbgm

In [7]:
import pyarrow.parquet as pq
import lightgbm as lgb
import numpy as np
import os
from glob import glob
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# === CONFIGURATION ===
CLEANED_DIR = r"D:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\data\cleaned"
MODEL_PATH = "final_lightgbm.txt"
BATCH_SIZE = 100_000

file_list = sorted(glob(os.path.join(CLEANED_DIR, "*.parquet")))
if len(file_list) < 3:
    raise Exception("Need at least 3 files for train/val/test split!")
train_files = file_list[:-2]
val_file = file_list[-2]
test_file = file_list[-1]

print("Train files:", train_files)
print("Validation file:", val_file)
print("Test file:", test_file)

print("\nCalculating scale_pos_weight from train files...")
total_0, total_1 = 0, 0
for f in train_files:
    parquet = pq.ParquetFile(f)
    for batch in parquet.iter_batches(batch_size=BATCH_SIZE, columns=['Label']):
        arr = batch.to_pandas()['Label'].value_counts()
        total_0 += arr.get(0, 0)
        total_1 += arr.get(1, 0)

scale_pos_weight = total_0 / total_1
print(f"scale_pos_weight: {scale_pos_weight:.3f}")

sample = pq.read_table(train_files[0], columns=None).to_pandas().iloc[:1]
feature_cols = [col for col in sample.columns if col != 'Label']

params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'learning_rate': 0.02,
    'num_leaves': 64,
    'max_depth': 7,
    'min_child_samples': 40,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'scale_pos_weight': scale_pos_weight,
    'verbose': -1
}

print("\nStarting batch-wise training with early stopping...")
val_parquet = pq.ParquetFile(val_file)
val_batch = next(val_parquet.iter_batches(batch_size=BATCH_SIZE))
df_val = val_batch.to_pandas()
X_val = df_val[feature_cols].astype(np.float32)
y_val = df_val['Label'].astype(np.int32)
lgb_val = lgb.Dataset(X_val, y_val, free_raw_data=False)  # <--- Fix here

booster = None
for file_path in train_files:
    print(f"\nProcessing: {file_path}")
    parquet_file = pq.ParquetFile(file_path)
    batch_idx = 1
    for batch in parquet_file.iter_batches(batch_size=BATCH_SIZE):
        df = batch.to_pandas()
        X = df[feature_cols].astype(np.float32)
        y = df['Label'].astype(np.int32)
        lgb_train = lgb.Dataset(X, y, free_raw_data=False)  # <--- Fix here
        if booster is None:
            print("  Training new model with early stopping …")
            booster = lgb.train(
                params,
                lgb_train,
                num_boost_round=500,
                valid_sets=[lgb_val],
                valid_names=['validation'],
                callbacks=[lgb.early_stopping(25), lgb.log_evaluation(25)]
            )
        else:
            print(f"  Continuing training … (batch {batch_idx})")
            booster = lgb.train(
                params,
                lgb_train,
                num_boost_round=500,
                init_model=booster,
                valid_sets=[lgb_val],
                valid_names=['validation'],
                callbacks=[lgb.early_stopping(25), lgb.log_evaluation(25)]
            )
        del df, X, y, lgb_train
        batch_idx += 1

booster.save_model(MODEL_PATH)
print("\nModel saved as:", MODEL_PATH)

print(f"\nEvaluating on: {test_file}")
test_parquet = pq.ParquetFile(test_file)
y_true_all, y_pred_prob = [], []
for batch in test_parquet.iter_batches(batch_size=BATCH_SIZE):
    df = batch.to_pandas()
    X = df[feature_cols].astype(np.float32)
    y = df['Label'].astype(np.int32)
    y_prob = booster.predict(X)
    y_true_all.extend(y.tolist())
    y_pred_prob.extend(y_prob.tolist())
    del df, X, y, y_prob

thresholds = [0.5, 0.4, 0.3, 0.2, 0.1, 0.05, 0.01]
for threshold in thresholds:
    y_pred = (np.array(y_pred_prob) > threshold).astype(int)
    print(f"\n=== Results for threshold={threshold} ===")
    print("Accuracy:", accuracy_score(y_true_all, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true_all, y_pred))
    print("Classification Report:\n", classification_report(y_true_all, y_pred, digits=4))

print("\nDone! The model and thresholded evaluation results are printed.")


Train files: ['D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thursday-01-03-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thursday-15-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\

### Using optuna (Tuning)

In [8]:
import optuna
import pyarrow.parquet as pq
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Path to one medium/large cleaned file
tune_file = r"D:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\data\cleaned\Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet"
BATCH_SIZE = 100_000

# Load a batch
parquet = pq.ParquetFile(tune_file)
for batch in parquet.iter_batches(batch_size=BATCH_SIZE):
    df = batch.to_pandas()
    break  # Take first batch

feature_cols = [c for c in df.columns if c != "Label"]
X = df[feature_cols].astype(np.float32)
y = df["Label"].astype(np.int32)

# Split for tuning
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

def objective(trial):
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'metric': 'auc',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 15, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 4.0, 8.0),  # Use around your computed value
        'verbose': -1,
    }
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)
    booster = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_val],
        valid_names=['validation'],
        num_boost_round=300,
        callbacks=[lgb.early_stopping(25), lgb.log_evaluation(0)]
    )
    # Use lower threshold to favor recall
    y_pred = (booster.predict(X_val) > 0.2).astype(int)
    return f1_score(y_val, y_pred, average='binary')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)  # 40-50 for reasonable coverage

best_params = study.best_params
print("Best hyperparameters:", best_params)


d:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-05 19:13:56,685] A new study created in memory with name: no-name-663e31c4-3b36-4269-b4c4-34b56448bee2
C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:34: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_f

Training until validation scores don't improve for 25 rounds


[I 2025-06-05 19:13:57,569] Trial 0 finished with value: 0.9999164508313142 and parameters: {'learning_rate': 0.04560353410164083, 'num_leaves': 101, 'max_depth': 20, 'min_child_samples': 95, 'subsample': 0.7884208766149358, 'colsample_bytree': 0.6499557736337581, 'scale_pos_weight': 4.303589271823835}. Best is trial 0 with value: 0.9999164508313142.


Early stopping, best iteration is:
[58]	validation's auc: 1
Training until validation scores don't improve for 25 rounds


C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:34: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
C:\Users\aroma

...

Training until validation scores don't improve for 25 rounds
Early stopping, best iteration is:
[14]	validation's auc: 1


C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:34: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
C:\Users\aroma

Training until validation scores don't improve for 25 rounds
Early stopping, best iteration is:
[11]	validation's auc: 0.999951


C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:34: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
C:\Users\aroma\AppData\Local\Temp\ipykernel_32672\58112322.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
C:\Users\aroma

Training until validation scores don't improve for 25 rounds
Early stopping, best iteration is:
[4]	validation's auc: 0.999578
Best hyperparameters: {'learning_rate': 0.1614458587407259, 'num_leaves': 150, 'max_depth': 19, 'min_child_samples': 74, 'subsample': 0.6628764303318666, 'colsample_bytree': 0.9327214397233908, 'scale_pos_weight': 5.72295277674116}


In [1]:
# try

import pyarrow.parquet as pq
import lightgbm as lgb
import numpy as np
import os
from glob import glob
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# === CONFIGURATION ===
CLEANED_DIR = r"D:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\data\cleaned"
MODEL_PATH = "final_lightgbm_tuned.txt"
BATCH_SIZE = 100_000  # Tune for your RAM

# 1. Auto-detect all cleaned parquet files, split train/test (last one as test for demo)
file_list = sorted(glob(os.path.join(CLEANED_DIR, "*.parquet")))
train_files = file_list[:-1]
test_file = file_list[-1]

print("Train files:", train_files)
print("Test file:", test_file)

# 2. Pick feature columns (from any file)
sample = pq.read_table(train_files[0], columns=None).to_pandas().head(1)
feature_cols = [col for col in sample.columns if col != 'Label']

# 3. Compute global scale_pos_weight (class 0 / class 1)
total_0 = 0
total_1 = 0
for f in train_files:
    df = pq.read_table(f, columns=['Label']).to_pandas()
    total_0 += (df['Label'] == 0).sum()
    total_1 += (df['Label'] == 1).sum()
scale_pos_weight = round(total_0 / total_1, 2)
print(f"scale_pos_weight: {scale_pos_weight}")

# 4. LightGBM parameters (tuned)
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'learning_rate': 0.03,      # Slower learning for better results
    'num_leaves': 40,           # Try between 32-64 for large data
    'max_depth': 10,            # Max tree depth
    'scale_pos_weight': scale_pos_weight,  # Handle imbalance
    'feature_fraction': 0.8,    # Randomly select features at each tree
    'bagging_fraction': 0.8,    # Randomly select data at each iteration
    'bagging_freq': 2,          # Perform bagging every 2 iterations
    'min_child_samples': 30,    # Minimum samples per leaf
    'verbose': -1,
    'n_jobs': -1
}

# === TRAINING ===
booster = None
for file_idx, file_path in enumerate(train_files):
    print(f"\nProcessing: {file_path}")
    parquet_file = pq.ParquetFile(file_path)
    batch_idx = 1
    for batch in parquet_file.iter_batches(batch_size=BATCH_SIZE):
        df = batch.to_pandas()
        X = df[feature_cols].astype(np.float32)
        y = df['Label'].astype(np.int32)
        lgb_train = lgb.Dataset(X, y, free_raw_data=True)
        if booster is None:
            print("  Training new model …")
            booster = lgb.train(
                params,
                lgb_train,
                num_boost_round=200,  # More boosting rounds
                valid_sets=[lgb_train],
                callbacks=[lgb.log_evaluation(period=25)],
                keep_training_booster=True
            )
        else:
            print(f"  Continuing training … (batch {batch_idx})")
            booster = lgb.train(
                params,
                lgb_train,
                num_boost_round=200,
                valid_sets=[lgb_train],
                init_model=booster,
                callbacks=[lgb.log_evaluation(period=25)],
                keep_training_booster=True
            )
        del df, X, y, lgb_train
        batch_idx += 1

# Save model
booster.save_model(MODEL_PATH)
print("\nModel saved as:", MODEL_PATH)

# === EVALUATION ===
print(f"\nEvaluating on: {test_file}")
test_parquet = pq.ParquetFile(test_file)
y_true_all, y_pred_all = [], []
for batch in test_parquet.iter_batches(batch_size=BATCH_SIZE):
    df = batch.to_pandas()
    X = df[feature_cols].astype(np.float32)
    y = df['Label'].astype(np.int32)
    y_pred_proba = booster.predict(X)
    y_pred = (y_pred_proba > 0.4).astype(int)  # Lower threshold to catch more attacks (try 0.4, 0.3, etc.)
    y_true_all.extend(y.tolist())
    y_pred_all.extend(y_pred.tolist())
    del df, X, y, y_pred

# Results
acc = accuracy_score(y_true_all, y_pred_all)
cm = confusion_matrix(y_true_all, y_pred_all)
cr = classification_report(y_true_all, y_pred_all, digits=4)

print("\n=== Test Results (threshold=0.4) ===")
print("Accuracy:", acc)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", cr)

with open("classification_report_lgbm_tuned.txt", "w") as f:
    f.write("Accuracy: {:.4f}\n".format(acc))
    f.write("Confusion Matrix:\n{}\n".format(cm))
    f.write("Classification Report:\n{}\n".format(cr))

print("\nDone! The tuned model and report are saved.")


Train files: ['D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-16-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Friday-23-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thursday-01-03-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\\data\\cleaned\\Thursday-15-02-2018_TrafficForML_CICFlowMeter.parquet', 'D:\\Canada\\Subjects\\Semester -2\\AIDI-2005-02 CAPSTONE TERM ll\\Bot_detector\

In [ ]:
# Sample try

In [8]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import optuna

# File paths
data_path = r'D:\Canada\Subjects\Semester -2\AIDI-2005-02 CAPSTONE TERM ll\Bot_detector\data\cleaned\*.parquet'
all_files = glob.glob(data_path)

# Sample data for prototyping/tuning (10,000 rows from each file)
df_list = []
for file in all_files:
    df_temp = pd.read_parquet(file)
    if len(df_temp) > 10000:
        df_temp = df_temp.sample(n=10000, random_state=42)
    df_list.append(df_temp)
df = pd.concat(df_list, ignore_index=True)
print("Combined sampled shape:", df.shape)

# Convert labels to binary
df['Label'] = df['Label'].replace({'Benign': 0, 'Attack': 1}).astype(int)

# Select numeric columns for features
X = df.drop(columns=['Label']).select_dtypes(include=np.number)
y = df['Label']


Combined sampled shape: (100000, 79)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [11]:
def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        "max_depth": trial.suggest_int("max_depth", 4, 16),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 100),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
        "scale_pos_weight": float(np.sum(y_train == 0)) / np.sum(y_train == 1),  # handle imbalance
    }
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_test, label=y_test)
    booster = lgb.train(
        params, dtrain, valid_sets=[dval],
        num_boost_round=200,
        callbacks=[lgb.early_stopping(stopping_rounds=20), lgb.log_evaluation(0)]  # disables logging, use 100 for every 100 rounds
    )

    preds = booster.predict(X_test)
    auc = roc_auc_score(y_test, preds)
    return auc

# Run Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)
print("Best params:", study.best_params)


[I 2025-06-16 13:38:56,701] A new study created in memory with name: no-name-68c9f722-33ac-4d96-921c-b7c270b381fb


Training until validation scores don't improve for 20 rounds


[I 2025-06-16 13:38:59,079] Trial 0 finished with value: 0.9808502718898113 and parameters: {'learning_rate': 0.18139066159802691, 'num_leaves': 117, 'max_depth': 11, 'feature_fraction': 0.888821100941702, 'bagging_fraction': 0.5528380922327147, 'bagging_freq': 1, 'min_data_in_leaf': 75, 'lambda_l1': 3.911840621886989, 'lambda_l2': 0.20239210830115262}. Best is trial 0 with value: 0.9808502718898113.


Early stopping, best iteration is:
[49]	valid_0's auc: 0.98085
Training until validation scores don't improve for 20 rounds


[I 2025-06-16 13:39:00,545] Trial 1 finished with value: 0.9818915813012655 and parameters: {'learning_rate': 0.1254389560847368, 'num_leaves': 78, 'max_depth': 16, 'feature_fraction': 0.6147783077776022, 'bagging_fraction': 0.9960620687306792, 'bagging_freq': 9, 'min_data_in_leaf': 45, 'lambda_l1': 1.0974257818029893, 'lambda_l2': 2.726243314051949}. Best is trial 1 with value: 0.9818915813012655.


...

Did not meet early stopping. Best iteration is:
[200]	valid_0's auc: 0.980895
Training until validation scores don't improve for 20 rounds


[I 2025-06-16 13:40:58,043] Trial 28 finished with value: 0.9820798136089323 and parameters: {'learning_rate': 0.10065188835482072, 'num_leaves': 91, 'max_depth': 13, 'feature_fraction': 0.9321457434267331, 'bagging_fraction': 0.9690578237315238, 'bagging_freq': 2, 'min_data_in_leaf': 78, 'lambda_l1': 3.74392153379715, 'lambda_l2': 2.255676162023393}. Best is trial 21 with value: 0.9821931053312705.


Early stopping, best iteration is:
[103]	valid_0's auc: 0.98208
Training until validation scores don't improve for 20 rounds


[I 2025-06-16 13:41:05,759] Trial 29 finished with value: 0.9812666291197589 and parameters: {'learning_rate': 0.05208865322098913, 'num_leaves': 106, 'max_depth': 11, 'feature_fraction': 0.7193254471863219, 'bagging_fraction': 0.6116015187728692, 'bagging_freq': 4, 'min_data_in_leaf': 95, 'lambda_l1': 4.098335408378678, 'lambda_l2': 0.4990906467340883}. Best is trial 21 with value: 0.9821931053312705.


Early stopping, best iteration is:
[99]	valid_0's auc: 0.981267
Best params: {'learning_rate': 0.06036389701688897, 'num_leaves': 108, 'max_depth': 12, 'feature_fraction': 0.7315057258291888, 'bagging_fraction': 0.9452917185973672, 'bagging_freq': 5, 'min_data_in_leaf': 86, 'lambda_l1': 3.9343696523899667, 'lambda_l2': 1.1977201179904}


In [13]:
# Use best params and train final model
best_params = study.best_params
best_params["objective"] = "binary"
best_params["metric"] = "auc"
best_params["verbosity"] = -1

dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_test, label=y_test)
booster = lgb.train(
    best_params,
    dtrain,
    valid_sets=[dval],
    num_boost_round=200,
    callbacks=[
        lgb.early_stopping(stopping_rounds=20),
        lgb.log_evaluation(20)
    ]
)


# Save the model
booster.save_model('final_lightgbm_tuned.txt')


Training until validation scores don't improve for 20 rounds
[20]	valid_0's auc: 0.97785
[40]	valid_0's auc: 0.979886
[60]	valid_0's auc: 0.981022
[80]	valid_0's auc: 0.981625
[100]	valid_0's auc: 0.981836
[120]	valid_0's auc: 0.982084
[140]	valid_0's auc: 0.982041
Early stopping, best iteration is:
[122]	valid_0's auc: 0.982093


In [14]:
pred_probs = booster.predict(X_test)
threshold = 0.5
preds = (pred_probs > threshold).astype(int)

print("AUC:", roc_auc_score(y_test, pred_probs))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
print("Classification Report:\n", classification_report(y_test, preds))


AUC: 0.982093115734868
Confusion Matrix:
 [[15148    81]
 [  655  4116]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98     15229
           1       0.98      0.86      0.92      4771

    accuracy                           0.96     20000
   macro avg       0.97      0.93      0.95     20000
weighted avg       0.96      0.96      0.96     20000



In [15]:
booster.save_model('final_lightgbm_tuned.txt')


In [16]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

# 1. Load the test dataset
df = pd.read_parquet('D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned/Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet')  # Change path as needed

# 2. Encode the 'Label' column as 0 (Benign) and 1 (Attack)
df['Label'] = df['Label'].replace({'Benign': 0, 'Attack': 1}).astype(int)

# 3. Separate features and label
X_test = df.drop(columns=['Label'])
y_test = df['Label']

# 4. Optional: Keep only numeric columns
X_test = X_test.select_dtypes(include=np.number)

# 5. Load the pretrained LightGBM model
booster = lgb.Booster(model_file='final_lightgbm_tuned.txt')  # Change filename if different

# 6. Predict probabilities and classes
y_pred_probs = booster.predict(X_test)
y_pred = (y_pred_probs >= 0.5).astype(int)  # You can adjust threshold if needed

# 7. Evaluate the results
auc = roc_auc_score(y_test, y_pred_probs)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"AUC: {auc}")
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)

# 8. (Optional) Save results to CSV for further analysis
results = X_test.copy()
results['Actual'] = y_test
results['Predicted'] = y_pred
results['Pred_Prob'] = y_pred_probs
results.to_csv('test_results.csv', index=False)


AUC: 0.9999134772008106
Confusion Matrix:
 [[762237    147]
 [   322 285869]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    762384
           1       1.00      1.00      1.00    286191

    accuracy                           1.00   1048575
   macro avg       1.00      1.00      1.00   1048575
weighted avg       1.00      1.00      1.00   1048575



In [20]:
import numpy as np
import pandas as pd
import lightgbm as lgb

# 1. Feature order (from your list)
feature_names = [
    'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts',
    'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
    'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std',
    'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
    'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
    'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
    'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt',
    'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
    'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
    'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
    'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
    'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min'
]

# 2. Example attack values (based on common attack patterns, fill the rest as zeros if unsure)
attack_values = [
    80,    # Dst Port (common HTTP port)
    6,     # Protocol (TCP)
    200000, # Flow Duration (long connection)
    500,   # Tot Fwd Pkts
    1200,  # Tot Bwd Pkts
    100000, # TotLen Fwd Pkts
    300000, # TotLen Bwd Pkts
    1500,  # Fwd Pkt Len Max
    0, 0, 0,  # ... Min, Mean, Std (defaults)
    2000, 0, 0, 0,  # Bwd Pkt
    1e6, 200, 5000, 500, 9000, 1000, 50000, 1000, 12000, 100, 250000, 5000, 9000, 100, 500, 0, 0, 0, 0, 40, 40,
    10, 2000, 1200, 300, 80000, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 100, 500, 300, 200, 0, 0, 0, 0, 0, 0, 500, 50000, 200, 80000, 1024, 2048, 0, 40, 100, 0, 0, 0, 0, 0, 0
]
# Fill missing values with zeros if <78
if len(attack_values) < 78:
    attack_values += [0] * (78 - len(attack_values))

# 3. Convert to DataFrame
attack_df = pd.DataFrame([attack_values], columns=feature_names)

# 4. Load your pre-trained model
booster = lgb.Booster(model_file='final_lightgbm_tuned.txt')  # Change filename if needed

# 5. Predict
prob = booster.predict(attack_df)[0]
pred = int(prob >= 0.5)

print(f"Predicted probability of attack: {prob:.4f}")
print(f"Predicted class: {'Attack' if pred == 1 else 'Benign'}")


Predicted probability of attack: 0.1843
Predicted class: Benign


In [21]:
import pandas as pd
import numpy as np

# Load your data (already encoded with 0/1 in 'Label')
df = pd.read_parquet('D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned/Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet')

# Select feature columns (remove 'Label')
feature_cols = [col for col in df.columns if col != 'Label']

# 1. Sample a Benign flow
benign_sample = df[df['Label'] == 0].sample(n=1, random_state=42)[feature_cols]

# 2. Sample an Attack flow
attack_sample = df[df['Label'] == 1].sample(n=1, random_state=24)[feature_cols]


In [22]:
# Predict probability and class for Benign
benign_prob = booster.predict(benign_sample)[0]
benign_class = int(benign_prob >= 0.5)
print(f"Benign sample: Predicted probability of attack = {benign_prob:.4f} | Predicted class: {'Attack' if benign_class else 'Benign'}")

# Predict probability and class for Attack
attack_prob = booster.predict(attack_sample)[0]
attack_class = int(attack_prob >= 0.5)
print(f"Attack sample: Predicted probability of attack = {attack_prob:.4f} | Predicted class: {'Attack' if attack_class else 'Benign'}")


Benign sample: Predicted probability of attack = 0.0088 | Predicted class: Benign
Attack sample: Predicted probability of attack = 0.9967 | Predicted class: Attack


In [26]:
import pandas as pd
import numpy as np
import lightgbm as lgb

# -------- 1. Load Pretrained Model --------
booster = lgb.Booster(model_file='final_lightgbm_tuned.txt')

# -------- 2. Load Data --------
# Use the same file you trained on, or another file in same format
df = pd.read_parquet('D:/Canada/Subjects/Semester -2/AIDI-2005-02 CAPSTONE TERM ll/Bot_detector/data/cleaned/Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet')

# Make sure Label is encoded (for testing with real samples)
df['Label'] = df['Label'].replace({'Benign': 0, 'Attack': 1}).astype(int)

# Feature names as used for training
feature_names = [
    'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
    'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
    'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s',
    'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
    'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
    'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len',
    'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
    'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
    'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
    'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
    'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Win Byts',
    'Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
    'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min'
]

# -------- 3. Sample Real Benign & Attack --------

# Grab a real benign row
benign_row = df[df['Label'] == 0].iloc[0][feature_names].to_frame().T

# Grab a real attack row
attack_row = df[df['Label'] == 1].iloc[0][feature_names].to_frame().T

# -------- 4. Create Synthetic "Extreme Attack" --------

extreme_attack = pd.DataFrame([[
    8080,   # Dst Port (unusual)
    17,     # Protocol (UDP)
    800000, # Flow Duration
    5000,   # Tot Fwd Pkts
    9000,   # Tot Bwd Pkts
    2_000_000, # TotLen Fwd Pkts
    2_200_000, # TotLen Bwd Pkts
    *([1000] * (78 - 7))  # Make other features large (or adjust as you wish)
]], columns=feature_names)

# -------- 5. Predict --------

for name, sample in [('Benign (real)', benign_row),
                     ('Attack (real)', attack_row),
                     ('Attack (synthetic extreme)', extreme_attack)]:
    prob = booster.predict(sample)[0]
    pred = 'Attack' if prob >= 0.5 else 'Benign'
    print(f"{name} sample: Prob = {prob:.4f} | Predicted: {pred}")



Benign (real) sample: Prob = 0.1488 | Predicted: Benign
Attack (real) sample: Prob = 0.9980 | Predicted: Attack
Attack (synthetic extreme) sample: Prob = 0.0381 | Predicted: Benign


In [2]:
#!/usr/bin/env python3
"""
scripts/make_classifier_pkl.py

Loads your trained LightGBM model (final_lightgbm_tuned.txt),
wraps it in an sklearn‐compatible interface, and saves it
to app/models/classifier.pkl so your FastAPI and Kafka‐scorer
can load it via joblib.load().
"""

import os
import joblib
import numpy as np
import lightgbm as lgb

# 1) Load your LightGBM booster from disk
MODEL_FILE = "final_lightgbm_tuned.txt"
booster = lgb.Booster(model_file=MODEL_FILE)

# 2) Define the exact feature order you trained on:
feature_names = [
    'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
    'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
    'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
    'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s',
    'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
    'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
    'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
    'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags',
    'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s',
    'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
    'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
    'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
    'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
    'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
    'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts', 'Subflow Fwd Byts',
    'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
    'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
    'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min'
]

# 3) Wrap in an sklearn‐compatible interface
class LGBMWrapper:
    def __init__(self, booster, feature_names):
        self.booster = booster
        self.feature_names = feature_names

    def predict_proba(self, X):
        """
        X: pandas.DataFrame or 2D numpy array.
        Returns an (n_samples, 2) array: [[P(class0), P(class1)], ...].
        """
        # If it's a DataFrame, reorder columns
        try:
            Xmat = X[self.feature_names].values.astype(np.float32)
        except Exception:
            # assume it's already a numpy array
            Xmat = np.asarray(X, dtype=np.float32)
        p1 = self.booster.predict(Xmat)
        p0 = 1.0 - p1
        return np.vstack([p0, p1]).T

    def predict(self, X, threshold=0.5):
        """
        Returns binary predictions (0 or 1) for class1 at given threshold.
        """
        proba = self.predict_proba(X)[:, 1]
        return (proba >= threshold).astype(int)

# 4) Instantiate and dump to classifier.pkl
clf = LGBMWrapper(booster, feature_names)

# Ensure output directory exists
os.makedirs("app/models", exist_ok=True)

# Save with joblib so FastAPI can load via joblib.load()
joblib.dump(clf, "app/models/classifier.pkl")
print("✅ Wrote app/models/classifier.pkl")


✅ Wrote app/models/classifier.pkl


In [2]:
import pandas as pd
import numpy as np

N = 2000  # number of synthetic logins

np.random.seed(42)
data = {
    "time_to_submit": np.concatenate([
        np.random.uniform(1.5, 12, N//2),     # humans: slower
        np.random.uniform(0.02, 0.6, N//2)    # bots: very fast
    ]),
    "user_agent": np.random.choice([
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/119.0.0.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Firefox/118.0",
        "curl/8.5.0", "python-requests/2.28.1", "go-http-client/1.1"
    ], N),
    "login_hour": np.random.choice(range(24), N),
    "client_ip": np.random.choice([
        "203.0.113.1", "203.0.113.2", "45.77.1.3", "51.15.39.9", "192.0.2.1"
    ], N),
    "password_length": np.random.choice(range(6, 16), N),
    "failed_login_count_last_10min": np.concatenate([
        np.random.poisson(0.4, N//2),    # humans: low fails
        np.random.poisson(3.5, N//2)     # bots: many fails
    ]),
    "is_username_email": np.random.choice([0, 1], N),
    "label": np.concatenate([
        np.zeros(N//2),  # human
        np.ones(N//2)    # attack
    ])
}

df = pd.DataFrame(data)

# Optionally, shuffle the rows
df = df.sample(frac=1).reset_index(drop=True)

print(df.head())
df.to_csv("synthetic_login_data.csv", index=False)
print("\n✅ Saved to synthetic_login_data.csv")


   time_to_submit                                         user_agent  \
0        8.089360                                         curl/8.5.0   
1        0.220738                                         curl/8.5.0   
2        9.482393  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chro...   
3        0.507608  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chro...   
4        8.934762                                 go-http-client/1.1   

   login_hour    client_ip  password_length  failed_login_count_last_10min  \
0          19   51.15.39.9                9                              0   
1           8    45.77.1.3               14                              2   
2          16   51.15.39.9                6                              0   
3          12  203.0.113.2               14                              2   
4           5   51.15.39.9                7                              0   

   is_username_email  label  
0                  1    0.0  
1                  1    1.0  
2       